# Metodo guidotti
L'idea è quella di trasformare ogni traiettoria in una sequenza di simboli, estrarre i motif e usarli per classificare

In [61]:
import time
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import math
from datetime import timedelta, datetime
import plotly.express as px
from geolib import geohash
from sklearn.model_selection import train_test_split
from sklearn_extra.cluster import KMedoids


executor = ThreadPoolExecutor(max_workers=12)

df = pd.read_csv("vehicles_preapred.zip").sort_values(by=["tid", 't'])

df.tid -= df.tid.min()

df.c1 = df.c1/100000
df.c2 = df.c2/100000

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

df.head()

,tid,class,t,c1,c2
0,0,B,0,42.077160,4.738411
1,0,B,30,42.077246,4.739088
2,0,B,60,42.077259,4.739096
3,0,B,90,42.077369,4.739158
4,0,B,120,42.077635,4.739343


In [62]:
t = 0

In [63]:
%%time

start = datetime.now()

precision =  6 #da 1 a 12

df["geohash"] = [geohash.encode(x[0], x[1], precision) for x in tqdm(df[["c1", "c2"]].values)]

t += (datetime.now()-start).total_seconds()*1000

  0%|          | 0/178299 [00:00<?, ?it/s]

Wall time: 6.37 s


In [64]:
t

6367.136

In [65]:
%%time

start = datetime.now()

decodes = dict()

for gh in tqdm(df.geohash.unique()):
    decodes[gh] = geohash.bounds(gh)

l = []
prec = "!"
c = -1
for gh in df.geohash:
    if gh != prec:
        prec = gh
        c += 1
    l.append(gh+"_"+str(c))

df["geohash2"] = l

df["lat_norm"] = df.c1 - df.geohash.transform(lambda x: decodes[x].sw.lat)
df["lon_norm"] = df.c2 - df.geohash.transform(lambda x: decodes[x].sw.lon)

t += (datetime.now()-start).total_seconds()*1000

  0%|          | 0/1424 [00:00<?, ?it/s]

Wall time: 1.03 s


In [66]:
px.line_3d(df[df.tid == 1], x="t", y="c1", z="c2", color="geohash2")

In [67]:
px.line_3d(df[df.tid == 1], x="t", y="lat_norm", z="lon_norm", color="geohash2")

In [68]:
px.line(df[df.tid==1], x="lat_norm", y="lon_norm", color="geohash2")

In [69]:
px.line_3d(df[df.tid == 1], x="t", y="c1", z="c2")

In [70]:
%%time

start = datetime.now()

tid_train, tid_test, _, _ = train_test_split(df.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

df_train = df[df.tid.isin(tid_train)]
df_test = df[~df.tid.isin(tid_train)]

t+= (datetime.now()-start).total_seconds()*1000
df_train

Wall time: 620 ms


,tid,class,t,c1,c2,geohash,geohash2,lat_norm,lon_norm
652,11102,B,0,42.078687,4.782075,sp7rku,sp7rku_175,0.001050,0.003022
653,11102,B,34,42.077084,4.781617,sp7rkg,sp7rkg_176,0.004940,0.002564
654,11102,B,64,42.077367,4.780521,sp7rkg,sp7rkg_176,0.005223,0.001468
655,11102,B,94,42.077895,4.779303,sp7rku,sp7rku_177,0.000258,0.000250
656,11102,B,124,42.078541,4.778304,sp7rks,sp7rks_178,0.000904,0.010238
...,...,...,...,...,...,...,...,...,...
177632,389203,B,38516,42.084660,4.838882,sp7rqj,sp7rqj_50035,0.001530,0.004898
177633,389203,B,38546,42.082078,4.836761,sp7rqh,sp7rqh_50036,0.004441,0.002777
177634,389203,B,38576,42.080869,4.836100,sp7rqh,sp7rqh_50036,0.003232,0.002116
177635,389203,B,38606,42.078729,4.835051,sp7rqh,sp7rqh_50036,0.001092,0.001067


In [71]:
%%time

start = datetime.now()

movelets = []

movelet = []
movelets.append(movelet)
prec_gh2 = df_train.values[0, 6]
first_row = df_train.values[0]
for row in tqdm(df_train.values):
    if row[6] != prec_gh2:
        prec_gh2 = row[6]
        first_row = row
        movelet = []
        movelets.append(movelet)

    movelet.append((row[-2]-first_row[-2], row[-1]-first_row[-1], row[2]-first_row[2]))

t+= (datetime.now()-start).total_seconds()*1000

  0%|          | 0/125348 [00:00<?, ?it/s]

Wall time: 383 ms


In [72]:
import csv

with open("movelet veicoli guidotti 5.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerows(movelets)

In [73]:
#From here credits to Riccardo Guidotti

def spherical_distance(a, b):
    lat1 = a[1]
    lon1 = a[0]
    lat2 = b[1]
    lon2 = b[0]
    R = 6371000
    rlon1 = lon1 * math.pi / 180.0
    rlon2 = lon2 * math.pi / 180.0
    rlat1 = lat1 * math.pi / 180.0
    rlat2 = lat2 * math.pi / 180.0
    dlon = (rlon1 - rlon2) / 2.0
    dlat = (rlat1 - rlat2) / 2.0
    lat12 = (rlat1 + rlat2) / 2.0
    sindlat = math.sin(dlat)
    sindlon = math.sin(dlon)
    cosdlon = math.cos(dlon)
    coslat12 = math.cos(lat12)
    f = sindlat * sindlat * cosdlon * cosdlon + sindlon * sindlon * coslat12 * coslat12
    f = math.sqrt(f)
    f = math.asin(f) * 2.0 # the angle between the points
    f *= R
    return f

def trajectory_distance(tr1, tr2):
    if len(tr1) == 1 and len(tr2) == 1:
        return .0

    i1 = 0
    i2 = 0
    np = 0

    last_tr1 = tr1[i1]
    last_tr2 = tr2[i2]

    dist = spherical_distance(last_tr1, last_tr2)
    np += 1

    while True:
        if i1 >= (len(tr1)-1) or i2 >= (len(tr2)-1):
            break

        step_tr1 = spherical_distance(last_tr1, tr1[i1 + 1])
        step_tr2 = spherical_distance(last_tr2, tr2[i2 + 1])

        if step_tr1 < step_tr2:
            i1 += 1
            last_tr1 = tr1[i1]
            last_tr2 = closest_point_on_segment(last_tr2, tr2[i2+1], last_tr1)
        elif step_tr1 > step_tr2:
            i2 += 1
            last_tr2 = tr2[i2]
            last_tr1 = closest_point_on_segment(last_tr1, tr1[i1+1], last_tr2)
        else:
            i1 += 1
            i2 += 1
            last_tr1 = tr1[i1]
            last_tr2 = tr2[i2]

        d = spherical_distance(last_tr1, last_tr2)

        dist += d
        np += 1

    for i in range(i1, len(tr1)):
        d = spherical_distance(tr2[-1], tr1[i])
        dist += d
        np += 1

    for i in range(i2, len(tr2)):
        d = spherical_distance(tr1[-1], tr2[i])
        dist += d
        np += 1

    dist = 1.0 * dist / np

    return dist

def closest_point_on_segment(a, b, p):
    sx1 = a[0]
    sx2 = b[0]
    sy1 = a[1]
    sy2 = b[1]
    sz1 = a[2]
    sz2 = b[2]
    px = p[0]
    py = p[1]

    x_delta = sx2 - sx1
    y_delta = sy2 - sy1
    z_delta = sz2 - sz1

    if x_delta == 0 and y_delta == 0:
        return p

    u = ((px - sx1) * x_delta + (py - sy1) * y_delta) / (x_delta * x_delta + y_delta * y_delta)
    if u < 0:
        closest_point = a
    elif u > 1:
        closest_point = b
    else:
        cp_x = sx1 + u * x_delta
        cp_y = sy1 + u * y_delta
        dist_a_cp = spherical_distance(a, [cp_x, cp_y, 0])
        if dist_a_cp != 0:
            cp_z = sz1 + z_delta / (spherical_distance(a, b) / spherical_distance(a, [cp_x, cp_y, 0]))
        else:
            cp_z = a[2]
        closest_point = [cp_x, cp_y, cp_z]

    return closest_point

In [74]:
%%time

start = datetime.now()

colToDelete = 0

new_movelets = []

for i in range(len(movelets)):
    if(len(movelets[i]) < 5):
        colToDelete += 1
        continue

    new_movelets.append(movelets[i])

t+= (datetime.now()-start).total_seconds()*1000

Wall time: 41 ms


In [75]:
%%time

start = datetime.now()

dim = len(new_movelets)

dist_matrix = np.zeros((dim, dim))

for i, tr1 in enumerate(tqdm(new_movelets)):
    for j, tr2 in enumerate(new_movelets[:i+1]):
        dist = trajectory_distance(tr1, tr2)
        dist_matrix[i, j] = dist
        dist_matrix[j, i] = dist
        
t+= (datetime.now()-start).total_seconds()*1000

  0%|          | 0/8194 [00:00<?, ?it/s]

Wall time: 58min 4s


In [76]:
np.save("dist_matrix_6.npy", dist_matrix)

In [77]:
dist_matrix = np.load("dist_matrix_6.npy")

In [78]:
len(movelets[0])

1

In [79]:
t

3492815.4729999998

# QUI

In [80]:
t = 0

def symbolize(medoids, tr):
    i = 0
    sym = []
    while i < len(tr):
        best_medoid_id = np.inf
        best_medoid_score = np.inf
        for j, medoid in enumerate(medoids):
            score = trajectory_distance(medoid, list(map(lambda x: (x[0]-tr[i][0], x[1]-tr[i][1], x[2]-tr[i][2]), tr[i:i+len(medoid)])))
            if best_medoid_score > score:
                best_medoid_id = j
                best_medoid_score = score
        sym.append(best_medoid_id)
        i += len(medoids[best_medoid_id])

    return sym

def myContains(big, small):
    count = 0
    for i in range(len(big)):
        if big[i] == small[count]:
            count +=1
        else:
            count = 0
        if count == len(small):
            return True

    return False

In [81]:
from sklearn.feature_selection import mutual_info_classif

y=df.groupby(by="tid").max()["class"]

In [82]:
def guidotti(n_clust=20, sliding_w=3, top_k=200):
    kmedoids = KMedoids(n_clusters=n_clust, random_state=0, metric="precomputed").fit(dist_matrix)    
    medoids = [new_movelets[x] for x in kmedoids.medoid_indices_]
    trajectories = [[(x[3], x[4], x[2]) for x in df[df.tid == tid].values] for tid in df.tid.unique()]
    
    trajectories_sym=[]
    for trajectory in tqdm(trajectories, desc="symbolize"):
        trajectories_sym.append(symbolize(medoids, trajectory))
            
    trajectories_sym_train, _, y_sym_tr, _ = train_test_split(trajectories_sym,
                                                        y,
                                                        test_size=.3,
                                                        stratify=y,
                                                        random_state=3)
    
    subtrajectories_sym=[]
    for i, tr in enumerate(tqdm(trajectories_sym_train, desc="sliding w")):
        for j in range(0, len(tr)-sliding_w):
            subtrajectories_sym.append(tr[j:j+sliding_w])
    
    sliding_w *= 2
    for i, tr in enumerate(tqdm(trajectories_sym_train, desc="sliding w")):
        for j in range(0, len(tr)-sliding_w):
            subtrajectories_sym.append(tr[j:j+sliding_w])
    
    sliding_w *= 2
    for i, tr in enumerate(tqdm(trajectories_sym_train, desc="sliding w")):
        for j in range(0, len(tr)-sliding_w):
            subtrajectories_sym.append(tr[j:j+sliding_w])
    
    matr_sub = np.ones((len(trajectories_sym_train), len(subtrajectories_sym)))*np.NAN
    for i, traiettoria in enumerate(tqdm(trajectories_sym_train, desc="contains")):
        for j, pattern in enumerate(subtrajectories_sym):
            matr_sub[i][j] = 1 if myContains(traiettoria, pattern) else 0
    
    mi = mutual_info_classif(matr_sub, y_sym_tr)
    d = {i: x for i, x in enumerate(mi)}
    d = {k: v for k, v in sorted(d.items(), key=lambda item: -item[1])}
    
    res = []
    for key in list(d.keys())[:top_k]:
        res.append(subtrajectories_sym[key])
        
    matr = np.ones((len(trajectories_sym), len(res)))*np.NAN
    for i, traiettoria in enumerate(tqdm(trajectories_sym)):
        for j, pattern in enumerate(res):
            matr[i][j] = 1 if myContains(traiettoria, pattern) else 0
    
    return matr

In [83]:
def runTest(n_clust, sliding_w, top_k):
    start = datetime.now()
    
    X = guidotti(n_clust=n_clust, sliding_w=sliding_w, top_k=top_k)
    
    return (datetime.now()-start).total_seconds()*1000, X

In [84]:
def foo(x):
    return (x, x*x)

In [85]:
from concurrent.futures import ProcessPoolExecutor

executor = ProcessPoolExecutor(max_workers=1)

n_clusters = [2, 5, 10, 20, 100]
sliding_windows = [2, 3, 5, 10]
top_k = [1, 2, 10, 30, 50]

res_t = []
res_X = []

conf = []
processes = []
for n_clust in tqdm(n_clusters, position=0):
    for sliding_w in sliding_windows:
        for k in top_k:
            try:
                conf.append((n_clust, sliding_w, k))
                #processes.append(executor.submit(foo, 1))
                #processes.append(executor.submit(runTest, n_clust, sliding_w, k))
                t, x = runTest(n_clust, sliding_w, k)
                res_t.append(t)
                res_X.append(x)
            except:
                print("FAIL")
                res_t.append(np.NAN)
                res_X.append(np.NAN)


  0%|          | 0/5 [00:00<?, ?it/s]

symbolize:   0%|          | 0/381 [00:00<?, ?it/s]

sliding w:   0%|          | 0/266 [00:00<?, ?it/s]

sliding w:   0%|          | 0/266 [00:00<?, ?it/s]

sliding w:   0%|          | 0/266 [00:00<?, ?it/s]

...

sliding w:   0%|          | 0/266 [00:00<?, ?it/s]

sliding w:   0%|          | 0/266 [00:00<?, ?it/s]

sliding w:   0%|          | 0/266 [00:00<?, ?it/s]

contains:   0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/381 [00:00<?, ?it/s]

In [86]:
conf

[(2, 2, 1),
 (2, 2, 2),
 (2, 2, 10),
 (2, 2, 30),
 (2, 2, 50),
 (2, 3, 1),
 (2, 3, 2),
 (2, 3, 10),
 (2, 3, 30),
 (2, 3, 50),
 (2, 5, 1),
 (2, 5, 2),
 (2, 5, 10),
 (2, 5, 30),
 (2, 5, 50),
 (2, 10, 1),
 (2, 10, 2),
 (2, 10, 10),
 (2, 10, 30),
 (2, 10, 50),
 (5, 2, 1),
 (5, 2, 2),
 (5, 2, 10),
 (5, 2, 30),
 (5, 2, 50),
 (5, 3, 1),
 (5, 3, 2),
 (5, 3, 10),
 (5, 3, 30),
 (5, 3, 50),
 (5, 5, 1),
 (5, 5, 2),
 (5, 5, 10),
 (5, 5, 30),
 (5, 5, 50),
 (5, 10, 1),
 (5, 10, 2),
 (5, 10, 10),
 (5, 10, 30),
 (5, 10, 50),
 (10, 2, 1),
 (10, 2, 2),
 (10, 2, 10),
 (10, 2, 30),
 (10, 2, 50),
 (10, 3, 1),
 (10, 3, 2),
 (10, 3, 10),
 (10, 3, 30),
 (10, 3, 50),
 (10, 5, 1),
 (10, 5, 2),
 (10, 5, 10),
 (10, 5, 30),
 (10, 5, 50),
 (10, 10, 1),
 (10, 10, 2),
 (10, 10, 10),
 (10, 10, 30),
 (10, 10, 50),
 (20, 2, 1),
 (20, 2, 2),
 (20, 2, 10),
 (20, 2, 30),
 (20, 2, 50),
 (20, 3, 1),
 (20, 3, 2),
 (20, 3, 10),
 (20, 3, 30),
 (20, 3, 50),
 (20, 5, 1),
 (20, 5, 2),
 (20, 5, 10),
 (20, 5, 30),
 (20, 5, 50),
 (20,

In [87]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def compute_measures(test, pred):
    return (accuracy_score(test, pred), precision_score(test, pred, average="micro"), f1_score(test, pred, average="micro"), recall_score(test, pred, average="micro"))

res_val = []

for x in tqdm(res_X):
    try:

        X = x
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

        tree_para = {'n_estimators':range(100, 1500, 300),
                     'max_depth':range(2, 18, 3)}
        gs = GridSearchCV(RandomForestClassifier(), tree_para, cv=5, n_jobs=12, verbose=0)
        result = gs.fit(X_train, y_train)

        y_pred = result.best_estimator_.fit(X_train, y_train).predict(X_test)

        res_val.append(compute_measures(y_test, y_pred))

        print(compute_measures(y_test, y_pred))
    except:
        print("skip")
        res_val.append((np.NAN, np.NAN, np.NAN, np.NAN))

  0%|          | 0/100 [00:00<?, ?it/s]

(0.7130434782608696, 0.7130434782608696, 0.7130434782608696, 0.7130434782608696)
(0.7130434782608696, 0.7130434782608696, 0.7130434782608696, 0.7130434782608696)
(0.7217391304347827, 0.7217391304347827, 0.7217391304347827, 0.7217391304347827)
(0.7217391304347827, 0.7217391304347827, 0.7217391304347827, 0.7217391304347827)
(0.7304347826086957, 0.7304347826086957, 0.7304347826086957, 0.7304347826086957)
(0.7130434782608696, 0.7130434782608696, 0.7130434782608696, 0.7130434782608696)
(0.6956521739130435, 0.6956521739130435, 0.6956521739130435, 0.6956521739130435)
(0.7043478260869566, 0.7043478260869566, 0.7043478260869566, 0.7043478260869566)
(0.6869565217391305, 0.6869565217391305, 0.6869565217391305, 0.6869565217391305)
(0.6956521739130435, 0.6956521739130435, 0.6956521739130435, 0.6956521739130435)
(0.7130434782608696, 0.7130434782608696, 0.7130434782608696, 0.7130434782608696)
(0.7130434782608696, 0.7130434782608696, 0.7130434782608696, 0.7130434782608696)
(0.7043478260869566, 0.70434

In [89]:
df_res = pd.DataFrame(res_val, columns=["acc", "precision", "f1", "recall"])

df_res[["nclust", "sliding", "k"]] = conf

df_res["t"] = res_t

df_res["t"] += 3492*1000

df_res.to_csv("Vehicle search multiple rappresentation precision 6.csv", index=None)

df_res

,acc,precision,f1,recall,nclust,sliding,k,t
0,0.713043,0.713043,0.713043,0.713043,2,2,1,3546647.006
1,0.713043,0.713043,0.713043,0.713043,2,2,2,3547816.237
2,0.721739,0.721739,0.721739,0.721739,2,2,10,3546867.370
3,0.721739,0.721739,0.721739,0.721739,2,2,30,3547898.232
4,0.730435,0.730435,0.730435,0.730435,2,2,50,3548640.011
...,...,...,...,...,...,...,...,...
95,0.713043,0.713043,0.713043,0.713043,100,10,1,4144742.011
96,0.713043,0.713043,0.713043,0.713043,100,10,2,4149634.476
97,0.704348,0.704348,0.704348,0.704348,100,10,10,4162665.415
98,0.721739,0.721739,0.721739,0.721739,100,10,30,4153645.645


In [92]:
df_res.sort_values(by="acc", ascending=False).head()

,acc,precision,f1,recall,nclust,sliding,k,t
89,0.791304,0.791304,0.791304,0.791304,100,3,50,4313920.387
87,0.791304,0.791304,0.791304,0.791304,100,3,10,4242672.643
69,0.791304,0.791304,0.791304,0.791304,20,3,50,3676550.699
23,0.791304,0.791304,0.791304,0.791304,5,2,30,3557958.989
67,0.782609,0.782609,0.782609,0.782609,20,3,10,3675986.895


In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

tree_para = {'n_estimators':range(100, 1500, 300),
             'max_depth':range(2, 18, 3)}
gs = GridSearchCV(RandomForestClassifier(), tree_para, cv=10, n_jobs=12, verbose=3)
result = gs.fit(X_train, y_train)

report(gs.cv_results_, n_top=1)

Fitting 10 folds for each of 42 candidates, totalling 420 fits
Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 2, 'n_estimators': 400}

Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 2, 'n_estimators': 700}

Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 2, 'n_estimators': 1000}

Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 2, 'n_estimators': 1300}

Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 5, 'n_estimators': 100}

Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 5, 'n_estimators': 1000}

Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 5, 'n_estimators': 1300}

Model with rank: 1
Mean validation score: 0.726 (std: 0.052)
Parameters: {'max_depth': 5, 'n_estimators': 1600}

Model with rank: 1
Mean validation s

In [34]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=3, n_jobs=-1, n_estimators=400)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           B       1.00      0.09      0.17        33
           T       0.73      1.00      0.85        82

    accuracy                           0.74       115
   macro avg       0.87      0.55      0.51       115
weighted avg       0.81      0.74      0.65       115



In [37]:
print(classification_report(y_test, result.best_estimator_.fit(X_train, y_train).predict(X_test)))

              precision    recall  f1-score   support

           B       1.00      0.09      0.17        33
           T       0.73      1.00      0.85        82

    accuracy                           0.74       115
   macro avg       0.87      0.55      0.51       115
weighted avg       0.81      0.74      0.65       115

